In [89]:
import simpy
import random
import numpy as np
from scipy import stats

In [90]:
class FiFoServers:
    def __init__(self, env, mu, num_servers):
        self.mu = mu
        self.server = simpy.Resource(env, capacity=num_servers)
        self.processing_times = []
        self.final_arrival = 0
        self.N = 0

class SJFServers:
    def __init__(self, env, mu, num_servers):
        self.mu = mu
        self.server = simpy.PriorityResource(env, capacity=num_servers)
        self.processing_times = []
        self.final_arrival = 0
        self.N = 0

def task(name, env, servers, processing_time, waiting_times, print_tasks):

    # A task arrives at the server
    start_time = env.now
    if print_tasks:
        print(f'Task {name} arriving at {start_time}')
    
    servers.final_arrival = start_time
    servers.N += 1

    # Request task to server
    with servers.server.request() as req:

        # Once there is availability at the server, initiate task
        yield req
        end_time = env.now
        if print_tasks:
            print(f'Server starts processing {name} at {end_time}')

        yield env.timeout(processing_time)

        if print_tasks:
            print(f'Server done processing {name} at {env.now}')

        waiting_times.append(end_time - start_time)
        servers.processing_times.append(processing_time)


def task_generator(env, servers, N, lambda_, waiting_times, print_tasks=False):

    # Generate N tasks
    for i in range(N):
        
        # Calculate time at server by Poisson process, not sure if necessary
        processing_time = max(0, int(random.normalvariate(1 / servers.mu, 1)))

        # Create task
        env.process(task(i, env, servers, processing_time, waiting_times, print_tasks))

        # Calculate time until next task by Poisson process, not sure if necessary
        next_arrival = max(0, int(random.normalvariate(1 / lambda_, 1)))
        yield env.timeout(next_arrival)


In [91]:
def get_confidence(data, p=0.95):
    """
    Calculate the mean and confidence interval of a dataset for a given confidence level.

    Parameters:
    - data (array-like): Input data.
    - p (float, optional): Confidence level (default is 0.95).

    Returns:
    tuple: A tuple containing the mean and the confidence interval.
    """

    mean = np.mean(data)
    n = len(data)
    lamb = stats.t.ppf((1 + p) / 2, n - 1)
    sigma = np.std(data)
    confidence = (lamb * sigma / np.sqrt(n))
    return mean, confidence

In [92]:
def run_simulation(num_servers, mu, lambda_, N, tmax, scheduler):
    env = simpy.Environment()
    if scheduler.lower() == 'fifo':
        servers = FiFoServers(env, mu, num_servers)
    elif scheduler.lower() == 'sjf':
        servers = SJFServers(env, mu, num_servers)

    waiting_times = []

    env.process(task_generator(env, servers, N, lambda_ * num_servers, waiting_times))
    env.run(tmax)

    # Calculate statistics on waiting times
    waiting_times_array = np.array(waiting_times)
    std_waiting_time = np.std(waiting_times_array)
    
    confidence_level = 0.95

    mean, confidence = get_confidence(waiting_times_array, confidence_level)
    confidence_interval = (mean - confidence, mean + confidence)

    print(f"{num_servers} servers:")
    print(f"Average Waiting Time: {mean}")
    print(f"Standard Deviation of Waiting Time: {std_waiting_time}")
    print(f"Confidence Interval (95%): {confidence_interval}\n")

    # Calculate statistics on processing times
    processing_times_array = np.array(servers.processing_times)
    processing_rates_array = 1 / np.where(processing_times_array == 0, 0.001, processing_times_array)
    std_processing_time = np.std(processing_rates_array)

    mean, confidence = get_confidence(processing_rates_array, confidence_level)
    confidence_interval = (mean - confidence, mean + confidence)

    print(f"Average Processing Rate of server: {mean}")
    print(f"Standard deviation of Processing Rate {std_processing_time}")
    print(f"Confidence Interval (95%): {confidence_interval}\n")

    print(f"Time of final arrival: {servers.final_arrival}")
    print(f"Number of arrivals: {servers.N}")
    print(f"Average arrivals per timestep: {servers.N / servers.final_arrival}\n\n")


mu = 0.2
lambda_ = 0.15
N = 10000
tmax = 500000
num_servers_list = [1, 2, 4]
scheduler = 'fifo'

for num_servers in num_servers_list:
    run_simulation(num_servers, mu, lambda_, N, tmax, scheduler)

1 servers:
Average Waiting Time: 0.117
Standard Deviation of Waiting Time: 0.42108312718512003
Confidence Interval (95%): (0.10874592322002095, 0.12525407677997905)

Average Processing Rate of server: 0.2367409523809524
Standard deviation of Processing Rate 0.06788408055794289
Confidence Interval (95%): (0.23541028777605788, 0.23807161698584695)

Time of final arrival: 61592
Number of arrivals: 10000
Average arrivals per timestep: 0.16235874788933627


2 servers:
Average Waiting Time: 0.4925
Standard Deviation of Waiting Time: 1.012493827141677
Confidence Interval (95%): (0.4726530832062806, 0.5123469167937194)

Average Processing Rate of server: 0.23699369047619048
Standard deviation of Processing Rate 0.0725236484997079
Confidence Interval (95%): (0.2355720810018749, 0.23841529995050606)

Time of final arrival: 28497
Number of arrivals: 10000
Average arrivals per timestep: 0.35091413131206795


4 servers:
Average Waiting Time: 6.0654
Standard Deviation of Waiting Time: 6.602478537640

In [93]:
scheduler = 'sjf'

for num_servers in num_servers_list:
    run_simulation(num_servers, mu, lambda_, N, tmax, scheduler)

1 servers:
Average Waiting Time: 0.1055
Standard Deviation of Waiting Time: 0.3836270975830565
Confidence Interval (95%): (0.09798013678558298, 0.11301986321441701)

Average Processing Rate of server: 0.2369020634920635
Standard deviation of Processing Rate 0.07109356398639208
Confidence Interval (95%): (0.23550848655244877, 0.23829564043167822)

Time of final arrival: 61704
Number of arrivals: 10000
Average arrivals per timestep: 0.16206404771165564


2 servers:
Average Waiting Time: 0.5074
Standard Deviation of Waiting Time: 1.05401387087647
Confidence Interval (95%): (0.486739206784335, 0.5280607932156649)

Average Processing Rate of server: 0.23689797619047623
Standard deviation of Processing Rate 0.06710790912820577
Confidence Interval (95%): (0.23558252610775526, 0.2382134262731972)

Time of final arrival: 28202
Number of arrivals: 10000
Average arrivals per timestep: 0.35458478122118997


4 servers:
Average Waiting Time: 4.8956
Standard Deviation of Waiting Time: 6.2798487752492